# Aquifer Storage and Recovery with MODFLOW6
David Haasnoot, Justin van Beek

In [ ]:
# import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5, 3) # set default figure size
import flopy as fp  # import flopy and call it fp
import glob
import imageio
import IPython
from tqdm import tqdm
import warnings
import datetime
warnings.simplefilter("ignore")

Adjust number of layers to minimise model error

10 layers = ~5min \
15 layers = ~10min \
20 layers = ~15min \
25 layers = ~20<min \
30 layers = ~21 min  \
35...

In [ ]:
# nlay = 10 # number of layers
nlays = [20]
for nlay in tqdm(nlays):
    now = datetime.datetime.now()
    print(f'Start {nlay} layers at {now.hour}:{now.minute}')
    end = now + datetime.timedelta(minutes=(nlay-5))
    print(f'Expected end at {end.hour}:{end.minute}')
    Q_d = 40_000
    Q_tot = Q_d * 1.25 ## Change this later as we actually want to produce this, currentlty we still have 80% loss
    d_extrating = 62
    d_injecting = 365 - d_extrating ## change later for days with water excess
    print(f'Need to pump {Q_tot/d_extrating:.2f}m^3/d to full fill demand') 
    # domain size and boundary conditions
    R = 200 # length of domain, m
    hR = 0 # head at r=R
    hL = hR
    
    # aquifer parameters
    k = 30 # hydraulic conductivity, m/d -> between 10 - 40: start with 30
    H = 20 # aquifer thickness, m
    npor = 0.35 # porosity, generally 0.25- 0.5 
    
    # flow
    
    Q = Q_tot/d_extrating # extraction rate, m^3/d 
    Q_in = Q_tot / (d_injecting)# injection rate m^3/d
    
    # transport
    alphaL = 0.5 # longitudinal dispersivity in horizontal direction, m - ###################### something to check!! -> slides guest lecture
    alphaT = alphaL / 10 # transverse dispersivity in horizontal direction, m
    diffusion_coef = 0 # diffusion coeffcient
    
    # concentration
    cs = 30 # initial concentration, kg/m^3 (=g/L)
    cf = 0 # concentration injected water, kg/m^3 (=g/L)
    
    # buoyancy
    rhoref = 1000 # reference density, kg/m^3
    cref = 0 # reference concentration, kg/m^3
    drhodc = 0.7143  # Slope of the density-concentration line
    
    # space discretization
    delr = 0.2 # length of cell along row (in x-direction), m
    delc = 1 # width of cells normal to plane of flow (in y-direction), m
    
    
    z = np.linspace(0, -H, nlay + 1) # top and bottom(s) of layers
    zc = 0.5 * (z[:-1] + z[1:]) # center so cells, used for contouring
    nrow = 1 # number of rows
    ncol = round(R / delr) # number of columns
    rw = 0.2 # radius of well, m
    
    # convert parameters for radial flow following procedure of Langevin, 2008
    # radialize parameters:
    theta = 2 * np.pi
    r = np.cumsum(delr * np.ones(ncol)) - 0.5 * delr # rightside of cell minus half the cell length
    krad = k * r * theta * np.ones((nlay, nrow, ncol))
    nporrad = npor * r * theta * np.ones((nlay, nrow, ncol))
    
    # time discretization
    ########## injection
    tin = d_injecting # injection time, d - rest of the year - maybe change later
    delt = 0.1 # time step, d
    nstepin = round(tin / delt) # computed number of steps during injection, integer
    
    ######### extraction
    tout = d_extrating # extraction time, d
    delt = 0.1 # time step, d
    nstepout = round(tout / delt) # computed number of steps during extraction, integer
    
    # model name and workspace
    modelname = 'modelrad_nlay' # name of model
    gwfname = modelname + 'f' # name of flow model
    gwtname = modelname + 't' # name of transport model
    modelws = './' + modelname # model workspace to be used

    ### SIM ###
    # simulation
    sim = fp.mf6.MFSimulation(sim_name=modelname, # name of simulation
                              version='mf6', # version of MODFLOW
                              exe_name='../bin/mf6', # absolute path to MODFLOW executable
                              sim_ws=modelws, # path to workspace where all files are stored
                             )
    
    # time discretization
    tdis = fp.mf6.ModflowTdis(simulation=sim, # add to the simulation called sim (defined above)
                              time_units="DAYS", 
                              nper=2, # number of stress periods 
                              perioddata=[[tin, nstepin, 1],
                                          [tout, nstepout, 1]], # period length, number of steps, timestep multiplier
                             )

    ### Create groundwater flow model ###
    # groundwater flow model
    gwf = fp.mf6.ModflowGwf(simulation=sim, # add to simulation called sim
                            modelname=gwfname, # name of gwf model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwf_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation called sim
                                 filename=gwf.name + '.ims', # file name to store ims
                                 linear_acceleration="BICGSTAB", # use BIConjuGantGradientSTABalized method
                                 inner_dvclose=1e-6
                                )                                                                                                
    # register solver
    sim.register_ims_package(solution_file=gwf_ims, # name of iterative model solver instance
                             model_list=[gwf.name], # list with name of groundwater flow model
                            )   
    
    # discretization
    gwf_dis = fp.mf6.ModflowGwfdis(model=gwf, # add to groundwater flow model called gwf
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delr, 
                                   delc=delc, 
                                   top=z[0], 
                                   botm=z[1:], 
                                  )
    
    # aquifer properties
    gwf_npf  = fp.mf6.ModflowGwfnpf(model=gwf, 
                                    k=krad, # horizontal k value
                                    k33=krad / 10,
                                    save_flows=True, # save the flow for all cells
                                   )
        
    # initial condition
    gwf_ic = fp.mf6.ModflowGwfic(model=gwf, 
                                 strt=hR, # initial head used for iterative solution
                                )
    
    # wells
    wellin = []
    wellout = []
    for ilay in range(nlay):
        wellin.append([(ilay, 0, 0),  Q / nlay, cf])  # [(layer, row, col), U, concentration]
        wellout.append([(ilay, 0, 0),  -Q / nlay, cf]) # specified concentration is not used, but must be specified 
    wel_spd = {0: wellin, 1: wellout} # stress period data for periods 0 and 1
    gwf_wel = fp.mf6.ModflowGwfwel(model=gwf, 
                                   stress_period_data=wel_spd, 
                                   auxiliary=['CONCENTRATION'],
                                   pname='WEL1', # package name
                                  )
    
    # constant head 
    chd = []
    for ilay in range(nlay):
        chd.append([(ilay,  0,  ncol-1), hL, cs]) # [(layer, row, col), head, concentration]
    chd_spd  = {0: chd, 1: chd}    # Stress period data
    gwf_chd = fp.mf6.ModflowGwfchd(model=gwf, 
                                   stress_period_data=chd_spd, 
                                   auxiliary=['CONCENTRATION'],
                                   pname='CHD1', # package name
                                  )
    
    # buoyancy
    buy = fp.mf6.ModflowGwfbuy(model=gwf,
                               packagedata=[0, drhodc, cref, gwtname, 'CONCENTRATION'], # [conc 1 species - 0= salt, drhodc, cref, gwtname, name]
                               denseref=rhoref, # reference concentration
                               nrhospecies=1, # number of species
                               density_filerecord=f"{gwf.name}.dst", # file name
                               pname='BUY1', 
                              )
        
    # output control
    oc = fp.mf6.ModflowGwfoc(model=gwf, 
                             saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwfname}.cbc", # file name where all budget output is stored
                             head_filerecord=f"{gwfname}.hds", # file name where all head output is stored
                            )


    
    # groundwater transport model
    gwt = fp.mf6.ModflowGwt(simulation=sim, 
                            modelname=gwtname, # name of groundwater transport model
                           )
    
    # iterative model solver
    gwt_ims  = fp.mf6.ModflowIms(simulation=sim,
                                 filename=gwt.name + '.ims', # must be different than file name of gwf model ims
                                 linear_acceleration="BICGSTAB",
                                 inner_dvclose=1e-6
                                ) 
    sim.register_ims_package(solution_file=gwt_ims, 
                             model_list=[gwt.name],
                            )
    
    # discretization
    gwt_dis = fp.mf6.ModflowGwtdis(model=gwt, 
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delr, 
                                   delc=delc, 
                                   top=z[0], 
                                   botm=z[1:], 
                                  )
    
    # mobile storage and transfer
    gwt_sto = fp.mf6.ModflowGwtmst(model=gwt, 
                                   porosity=nporrad, # porosity
                                   save_flows=True,
                                  )
    
    # initial condition
    gwt_ic = fp.mf6.ModflowGwtic(model=gwt, 
                                 strt=cs, # initial concentration
                                ) 
    
    # source sink mixing
    sourcelist = [("WEL1", "AUX", "CONCENTRATION"), ("CHD1", "AUX", "CONCENTRATION")]
    ssm = fp.mf6.ModflowGwtssm(model=gwt, 
                               sources=sourcelist, 
                               save_flows=True,
                               pname='SSM1', 
                              )
    
    # advection
    adv = fp.mf6.ModflowGwtadv(model=gwt,  
                               scheme="TVD",  # use Total Variation Diminishing (TVD)
                               pname='ADV1',
                              )
    
    # dispersion
    dsp = fp.mf6.ModflowGwtdsp(model=gwt, 
                               alh=alphaL,
                               ath1=alphaT, 
                               diffc=diffusion_coef,
                               pname='DSP1', 
                              )
    
    # output control
    oc = fp.mf6.ModflowGwtoc(model=gwt,
                             saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwtname}.cbc", # file name where all budget output is stored
                             concentration_filerecord=f"{gwtname}.ucn", # file name where all concentration output is stored
                            )
    fp.mf6.ModflowGwfgwt(simulation=sim, 
                     exgtype="GWF6-GWT6", 
                     exgmnamea=gwf.name , 
                     exgmnameb=gwt.name , 
                     filename=f"{modelname}.gwfgwt",
                    );
    sim.write_simulation(silent=True)
    success, _ = sim.run_simulation(silent=True) 
    if success == 1:
        print('Model solved successfully')
    else:
        print('Solve failed')
    cobj = gwt.output.concentration() # get handle to binary concentration file
    c = cobj.get_alldata().squeeze() # get the concentration data from the file
    times = np.array(cobj.get_times()) # get the times and convert to array

    climit = 1 # limit concentration, g/L

    ### compute loss ###
    itime_out = np.abs(times - tin).argmin()
    lst_delM1 = []
    lst_delM2 = []
    for layer_i in range(nlay):
        delM1 = np.zeros(len(times))
        delM2 = np.zeros(len(times))
        for index, t in enumerate(times):
            itime = np.abs(times - t).argmin() # find index of value in times closest to t
            # skip first timestep as itime-1 doesn't exisit
            if itime == 0:
                delM1[itime] = np.nan
                delM2[itime] = np.nan
            else:
                # change based on in flow or outflow 
                if itime <= itime_out:
                    delM1[index] = (Q/nlay * c[itime, layer_i, 0] -  Q/nlay * c[itime, layer_i, -1]) * delt
                else:
                    delM1[index] = (- Q/nlay * c[itime, layer_i, 0] + Q/nlay * c[itime, layer_i, -1]) * delt
                delM2_i = 0
                for index_r, r_i in enumerate(r):
                    radius_centre = r_i
                    radius_left = radius_centre - 0.5 * delr
                    radius_right = radius_centre + 0.5 * delr
                    delM2_i += (c[itime,layer_i,index_r] - c[itime - 1,layer_i,index_r]) * (radius_right**2 - radius_left**2) * H/nlay * np.pi * npor
                delM2[index] = delM2_i
            
        lst_delM1.append(delM1)
        lst_delM2.append(delM2)    

    delM1_sum = np.vstack(lst_delM1).sum(axis=0)
    delM2_sum = np.vstack(lst_delM2).sum(axis=0)
    plt.figure()
    plt.plot(times, delM1_sum, label=r'$\Delta M_1$')
    plt.plot(times, delM2_sum, label=r'$\Delta M_2$',ls="--")
    plt.legend()
    plt.title(f'Total mass balance for {nlay}')
    plt.xlabel('time (days)')
    plt.ylabel('Change in mass (kg)');
    plt.grid()
    plt.savefig(rf'figures\difference_in_total_mass_balance_for_{nlay}_layers.png',bbox_inches="tight")
    print(f'In the last timestep of extraction the mass balance is {delM1_sum[-1]:.2f}kg for ΔM1 & {delM2_sum[-1]:.2f}kg for ΔM2')
    plt.figure()
    diff = delM1_sum - delM2_sum
    plt.plot(times, diff, label="Difference between ΔM1 & ΔM2 over time")
    plt.legend()
    plt.title(f'Difference in total mass balance for {nlay}')
    plt.xlabel('time (days)')
    plt.ylabel('Difference in change in mass (kg)');
    plt.grid()
    plt.savefig(rf'figures\difference_in_total_mass_balance_for_{nlay}_layers.png',bbox_inches="tight")

In [ ]:
plt.figure()
diff = delM1_sum - delM2_sum
n = 0
plt.plot(times[n:], diff[n:], label="Difference between ΔM1 & ΔM2 over time")
plt.legend()
plt.title(f'Difference in total mass balance for {nlay}')
plt.xlabel('time (days)')
plt.ylabel('Difference in change in mass (kg)');
plt.grid()
plt.savefig(rf'figures\difference_in_total_mass_balance_for_{nlay}_layers.png',bbox_inches="tight")

In [ ]:
plt.figure()
diff = delM1_sum - delM2_sum
n = 5
plt.plot(times[n:], diff[n:], label="Difference between ΔM1 & ΔM2 over time")
plt.legend()
plt.title(f'Difference in total mass balance for {nlay}')
plt.xlabel('time (days)')
plt.ylabel('Difference in change in mass (kg)');
plt.grid()
plt.savefig(rf'figures\difference_in_total_mass_balance_for_{nlay}_layers_cropped.png',bbox_inches="tight")

<!-- Plot the concentration vs. $r$ at 10 day intervals. Make separate plots for the injection and extraction periods. -->